In [ ]:
Lab work №2

Student name - Khrystyna

Student surname - Zyryanova

Group - CS-32

In [28]:
# --- Необхідні імпорти ---
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Встановлення насіння
RANDOM_SEED = 11

print("--- 📊 Завантаження та підготовка Star Dataset ---")

# 1. Завантаження та Розділення Даних
try:
    # Використовуйте шлях, який виявився успішним (наприклад, 'archive/Star3642_balanced.csv')
    file_path = 'archive/Star3642_balanced.csv' 
    data = pd.read_csv(file_path)
    print(f"Дані успішно завантажено з: {file_path}. Розмір: {data.shape}")
    
    # 💥 ВИПРАВЛЕНО: Використовуємо ФАКТИЧНІ НАЗВИ СТОВПЦІВ
    # Виключаємо 'SpType' (текстові дані) та 'Unnamed: 0', якщо він є
    features = ['Vmag', 'Plx', 'e_Plx', 'B-V', 'Amag'] 
    
    X_full = data[features]
    y_full = data['TargetClass'] # 💥 Цільова змінна тепер називається 'TargetClass'

    # Примітка: Оскільки класів може бути лише 2 (0 та 1), LabelEncoder не потрібен, 
    # але ми його використовуємо для уніфікації назв класів.
    y_encoded = y_full.values # Вже числові
    class_names = ['Class 0', 'Class 1'] # Припускаємо бінарну класифікацію
    print(f"Класи: {class_names} закодовані як {np.unique(y_encoded)}")
    
except Exception as e:
    print(f"\nКритична помилка: {e}. Перевірте ще раз шлях до файлу та назви стовпців.")
    raise

# Розділення даних
X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_encoded, test_size=0.3, random_state=RANDOM_SEED, stratify=y_encoded
)

# 2. Передобробка Даних (Масштабування)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\nМасштабування даних завершено.")
print("-" * 50)

--- 📊 Завантаження та підготовка Star Dataset ---
Дані успішно завантажено з: archive/Star3642_balanced.csv. Розмір: (3642, 7)

Фактичні стовпці у завантаженому файлі (для контролю):
['Vmag', 'Plx', 'e_Plx', 'B-V', 'SpType', 'Amag', 'TargetClass']

ПОМИЛКА KeyError: Здається, файл archive/Star3642_balanced.csv не містить необхідних стовпців.
Спробуйте інший файл або виправте список 'features' та 'y_full' відповідно до виводу.


KeyError: "None of [Index(['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift', 'plate', 'MJD',\n       'fiberID'],\n      dtype='object')] are in the [columns]"

In [14]:
def generate_classification_data(n_samples=200, kernels=None, kernel_classes=None,
                                 dispersion=0.2, n_outliers=10, random_seed=42):
    
    np.random.seed(random_seed)

    if kernels is None:
        kernels = [gaussian_kernel(center=[2, 2]),
                   gaussian_kernel(center=[-2, -2])]

    if kernel_classes is None:
        kernel_classes = list(range(len(kernels)))

    n_inliers = n_samples - n_outliers
    points_per_kernel = n_inliers // len(kernels)

    X_parts, y_parts = [], []
    for i, kernel in enumerate(kernels):
        class_label = kernel_classes[i]
        X_class = kernel(points_per_kernel).astype(float)
        X_class += np.random.randn(points_per_kernel, 2) * dispersion
        y_class = np.full(points_per_kernel, class_label)
        X_parts.append(X_class)
        y_parts.append(y_class)

    X_inliers = np.vstack(X_parts)
    y_inliers = np.hstack(y_parts)

    if n_outliers > 0:
        # Визначаємо межі існуючих даних
        x_min, x_max = X_inliers[:, 0].min(), X_inliers[:, 0].max()
        y_min, y_max = X_inliers[:, 1].min(), X_inliers[:, 1].max()
        padding = (max(x_max - x_min, y_max - y_min)) * 0.1

        # Генеруємо викиди в межах, трохи більших за існуючі дані
        X_outliers = np.random.uniform(min(x_min, y_min) - padding,
                                       max(x_max, y_max) + padding,
                                       size=(n_outliers, 2))
        
        # Викиди отримують випадкову мітку одного з існуючих класів
        n_classes = len(np.unique(kernel_classes))
        y_outliers = np.random.randint(0, n_classes, size=n_outliers)

        # Об'єднання
        X = np.vstack([X_inliers, X_outliers])
        y = np.hstack([y_inliers, y_outliers])
    else:
        X, y = X_inliers, y_inliers

    # Перемішування даних
    permutation = np.random.permutation(X.shape[0])
    X, y = X[permutation], y[permutation]

    return X, y

In [15]:
# --- Крок 3: Навчання та Тюнінг Моделей (Порівняння гіперпараметрів) ---

# 1. LogisticRegression Tuning
print("\n--- 1. LogisticRegression Tuning (Linear Model) ---")
lr_results = {}
for C_val in [0.1, 1.0, 10.0]:
    # C - обернена сила регуляризації (менше C -> сильніша регуляризація)
    model = LogisticRegression(C=C_val, solver='liblinear', random_state=RANDOM_SEED, multi_class='ovr', max_iter=1000)
    model.fit(X_train_scaled, y_train)
    
    # Використовуємо show_classification_report для оцінки на тестовому наборі
    print(f"\nКонфігурація: C={C_val}")
    show_classification_report(X_test_scaled, y_test, model, title=f"LR Performance (C={C_val})")
    
    # Зберігаємо результат для порівняння
    y_pred = model.predict(X_test_scaled)
    lr_results[C_val] = accuracy_score(y_test, y_pred)


# 2. SVC Tuning (RBF Kernel)
print("\n--- 2. SVC Tuning (Non-Linear Kernel) ---")
svc_results = {}
for C_val, gamma_val in [(1.0, 'scale'), (10.0, 'scale'), (1.0, 0.01)]:
    # C: регуляризація, Gamma: вплив однієї точки
    model = SVC(kernel='rbf', C=C_val, gamma=gamma_val, random_state=RANDOM_SEED)
    model.fit(X_train_scaled, y_train)
    
    print(f"\nКонфігурація: C={C_val}, Gamma={gamma_val}")
    show_classification_report(X_test_scaled, y_test, model, title=f"SVC Performance (C={C_val}, Gamma={gamma_val})")

    y_pred = model.predict(X_test_scaled)
    svc_results[f'C={C_val},G={gamma_val}'] = accuracy_score(y_test, y_pred)


# 3. RandomForestClassifier Tuning
print("\n--- 3. RandomForestClassifier Tuning (Ensemble Model) ---")
rf_results = {}
for n_est, max_d in [(50, 10), (200, 10), (100, None)]:
    # n_estimators: кількість дерев, max_depth: обмеження глибини
    model = RandomForestClassifier(n_estimators=n_est, max_depth=max_d, random_state=RANDOM_SEED, n_jobs=-1)
    model.fit(X_train_scaled, y_train)
    
    print(f"\nКонфігурація: n_est={n_est}, d={max_d}")
    show_classification_report(X_test_scaled, y_test, model, title=f"RF Performance (n_est={n_est}, d={max_d})")

    y_pred = model.predict(X_test_scaled)
    rf_results[f'N={n_est},D={max_d}'] = accuracy_score(y_test, y_pred)


--- 1. LogisticRegression Tuning (Linear Model) ---


NameError: name 'RANDOM_SEED' is not defined